# Social Vulnerability Cork - Copernicus data
Processing of the Copernicus data into census output areas

## Environment

### R Libraries
The relvant R libraries are imported in to the kernal:

In [1]:
# Load R libraries
 if(!require("pacman"))
     install.packages("pacman")
     library("pacman")

p_load("sf", "terra", "exactextractr")

print("Loaded Packages:")
p_loaded()

Loading required package: pacman



[1] "Loaded Packages:"


[1] "exactextractr" "terra"         "sf"            "pacman"

In [2]:
### Output directory

In [3]:
# create the pipeline directory if it does not exist
pipelineDir <- file.path("../..","2_pipeline","Cork","1c_Copernicus","2022")
if(!dir.exists(pipelineDir)){
    dir.create(pipelineDir, recursive = TRUE)
    print(paste0(pipelineDir, " created"))
}

# set country variable which is used within the output filename
country <- "Ireland"

[1] "../../2_pipeline/Cork/1c_Copernicus/2022 created"


## Process

In [4]:
# Setup

# Copernicus folder
copDir <- "../../0_data/Cork/Copernicus/"

# high res layers - MPERVIOUSNESS, TREE COVER DENSITY
highResLayers <- c("IMD","TCD")

# census output areas
censusOA <- st_read("../../0_data/Cork/SmallAreas/2022/SMALL_AREA_2022.shp")

#terraOptions(tempdir = pipelineDir)

Reading layer `SMALL_AREA_2022' from data source 
  `/Cities/0_data/Cork/SmallAreas/2022/SMALL_AREA_2022.shp' using driver `ESRI Shapefile'
Simple feature collection with 18919 features and 28 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 417471.5 ymin: 519663.7 xmax: 734481.1 ymax: 966896.3
Projected CRS: IRENET95 / Irish Transverse Mercator


In [5]:
# need to join copernicus tiles together
for(cop in highResLayers){
    print(paste0("Processing: ", cop))
    flush.console()

    #export path
    countryCopPath <- paste0(pipelineDir, "/", country, "_", cop, ".tif")

    # have the tiles been joined together already?
    countryCopCheck <- file.exists(countryCopPath)

    if(countryCopCheck){
        print(paste0(countryCopCheck, " already created"))
        flush.console()
    } else {
        # join the tiles together and export it

        # get the files
        tiles <- list.files(copDir, full.names = TRUE, recursive = TRUE, pattern=paste0(".*", cop, ".*\\.tif$"))  

        # create an image catalog
        ic <- terra::sprc(lapply(tiles, rast))

        # mosic the tiles
        icMosaic <- mosaic(ic, filename = countryCopPath, fun="max", overwrite=TRUE)
        #icMosaic <- merge(ic, filename = countryCopPath, overwrite=TRUE) # quicker
    }   
}

[1] "Processing: IMD"
[1] "Processing: TCD"                     
                                          

In [6]:
Sys.time()
censusOA_IMD <- censusOA

# Impervious surface
copRaster <- rast(paste0(pipelineDir, "/", country, "_", "IMD", ".tif"))

# #calculate the area of the raster cell (m2)
cellArea <- res(copRaster)[1]*res(copRaster)[2]

# #calculate the number of cells within each output area  
censusOA_IMD$cellArea_m2 <- exact_extract(copRaster, censusOA_IMD, 'count', coverage_area = TRUE, progress = TRUE)

# calculate the area of each output area that is impervious
censusOA_IMD$impArea_m2 <- exact_extract(copRaster, censusOA_IMD, 'sum', progress = TRUE)

# calculate the percentage of each output area that is impervious
censusOA_IMD$impPerc <- round((censusOA_IMD$impArea_m2/censusOA_IMD$cellArea_m2)*100, 3)

# calculate Z score
censusOA_IMD$impervious <- scale(censusOA_IMD$impPerc)

head(censusOA_IMD)
Sys.time()

[1] "2025-01-25 17:15:47 GMT"

Warning message in .local(x, y, ...):
“Polygons transformed to raster CRS (EPSG:3035)”
Cannot preload entire working area of 1395265641 cells with max_cells_in_memory = 3e+07. Raster values will be read for each feature individually.




 |======================================================================| 100%

Warning message in .local(x, y, ...):
“Polygons transformed to raster CRS (EPSG:3035)”
Cannot preload entire working area of 1395265641 cells with max_cells_in_memory = 3e+07. Raster values will be read for each feature individually.




 |======================================================================| 100%

Registered S3 method overwritten by 'geojsonsf':
  method        from   
  print.geojson geojson



OBJECTID SA_GUID_20                          
1 1        4c07d11e-0a4f-851d-e053-ca3ca8c0ca7f
2 2        4c07d11e-0a3a-851d-e053-ca3ca8c0ca7f
3 3        4c07d11e-0a4e-851d-e053-ca3ca8c0ca7f
4 4        4c07d11e-0a25-851d-e053-ca3ca8c0ca7f
5 5        4c07d11e-0a57-851d-e053-ca3ca8c0ca7f
6 6        4c07d11e-2df2-851d-e053-ca3ca8c0ca7f
  SA_GUID__1                           SA_PUB2011 SA_PUB2016 SA_PUB2022
1 4c07d11e-0a4f-851d-e053-ca3ca8c0ca7f 017010016  017010016  017010016 
2 4c07d11e-0a3a-851d-e053-ca3ca8c0ca7f 017010046  017010046  017010046 
3 4c07d11e-0a4e-851d-e053-ca3ca8c0ca7f 017010037  017010037  017010037 
4 4c07d11e-0a25-851d-e053-ca3ca8c0ca7f 017010005  017010005  017010005 
5 4c07d11e-0a57-851d-e053-ca3ca8c0ca7f 017010036  017010036  017010036 
6 4c07d11e-2df2-851d-e053-ca3ca8c0ca7f 017010009  017010009  017010009 
  SA_GEOGID_ SA_CHANGE_ SA_URBAN_A SA_URBAN_1 geometry                       ⋯
1 A017010016 0          1          Carlow     MULTIPOLYGON (((673146.5 67... ⋯
2 A017010046 4          1          Carlow     MULTIPOLYGON (((673956.3 67... ⋯
3 A017010037 0          1          Carlow     MULTIPOLYGON (((673720.6 67... ⋯
4 A017010005 0          1          Carlow     MULTIPOLYGON (((671923.3 67... ⋯
5 A017010036 0          1          Carlow     MULTIPOLYGON (((673711.8 67... ⋯
6 A017010009 4          1          Carlow     MULTIPOLYGON (((673488.5 67... ⋯
  COUNTY_ENG COUNTY_GAE  CSO_LEA SHAPE_Leng SHAPE_Area
1 CARLOW     Ceatharlach CARLOW  2123.346   204944.2  
2 CARLOW     Ceatharlach CARLOW  2891.271   288225.4  
3 CARLOW     Ceatharlach CARLOW  2436.986   208426.9  
4 CARLOW     Ceatharlach CARLOW  2887.091   348798.6  
5 CARLOW     Ceatharlach CARLOW  2844.410   429349.1  
6 CARLOW     Ceatharlach CARLOW  1058.290    59299.2  
  geometry                       cellArea_m2 impArea_m2 impPerc impervious
1 MULTIPOLYGON (((673146.5 67... 204991.3     75945.95  37.048   0.4175754
2 MULTIPOLYGON (((673956.3 67... 288290.4     57333.25  19.887  -0.2955289
3 MULTIPOLYGON (((673720.6 67... 208474.4     69274.20  33.229   0.2588815
4 MULTIPOLYGON (((671923.3 67... 348880.3    109519.96  31.392   0.1825472
5 MULTIPOLYGON (((673711.8 67... 429446.9    209251.55  48.726   0.9028403
6 MULTIPOLYGON (((673488.5 67...  59312.7     18759.59  31.628   0.1923539

[1] "2025-01-25 17:26:53 GMT"

In [7]:
#output the data as a geojson
st_write(censusOA_IMD, file.path(pipelineDir, "oa_IMD.geojson"), delete_dsn = TRUE)

Deleting source `../../2_pipeline/Cork/1c_Copernicus/2022/oa_IMD.geojson' failed
Writing layer `oa_IMD' to data source 
  `../../2_pipeline/Cork/1c_Copernicus/2022/oa_IMD.geojson' using driver `GeoJSON'
Writing 18919 features with 32 fields and geometry type Multi Polygon.


In [8]:
Sys.time()
censusOA_TCD <- censusOA
# TREE COVER DENSITY
# cell size = 100 m2, therefore a percentage value directly equals m2, i.e. 100% coverage = 100 m2 area of impervious surface

# data
copRaster <- rast(paste0(pipelineDir, "/", country, "_", "TCD", ".tif"))

# #calculate the area of the raster cell (m2)
cellArea <- res(copRaster)[1]*res(copRaster)[2]

# #calculate the number of cells within each output area 
censusOA_TCD$cellArea_m2 <- exact_extract(copRaster, censusOA_TCD, 'count', coverage_area = TRUE, progress = TRUE)

# calculate the area of each output area that has tree cover
censusOA_TCD$tcdArea_m2 <- exact_extract(copRaster, censusOA_TCD, 'sum', progress = TRUE)

# calculate the percentage of each output area that has tree cover
censusOA_TCD$tcdPerc <- round((censusOA_TCD$tcdArea_m2/censusOA_TCD$cellArea_m2)*100, 3)

# calculate Z score
censusOA_TCD$treeCover <- -scale(censusOA_TCD$tcdPerc)

head(censusOA_TCD)
Sys.time()

[1] "2025-01-25 17:30:29 GMT"

Warning message in .local(x, y, ...):
“Polygons transformed to raster CRS (EPSG:3035)”
Cannot preload entire working area of 1395265641 cells with max_cells_in_memory = 3e+07. Raster values will be read for each feature individually.




 |======================================================================| 100%

Warning message in .local(x, y, ...):
“Polygons transformed to raster CRS (EPSG:3035)”
Cannot preload entire working area of 1395265641 cells with max_cells_in_memory = 3e+07. Raster values will be read for each feature individually.




 |======================================================================| 100%

OBJECTID SA_GUID_20                          
1 1        4c07d11e-0a4f-851d-e053-ca3ca8c0ca7f
2 2        4c07d11e-0a3a-851d-e053-ca3ca8c0ca7f
3 3        4c07d11e-0a4e-851d-e053-ca3ca8c0ca7f
4 4        4c07d11e-0a25-851d-e053-ca3ca8c0ca7f
5 5        4c07d11e-0a57-851d-e053-ca3ca8c0ca7f
6 6        4c07d11e-2df2-851d-e053-ca3ca8c0ca7f
  SA_GUID__1                           SA_PUB2011 SA_PUB2016 SA_PUB2022
1 4c07d11e-0a4f-851d-e053-ca3ca8c0ca7f 017010016  017010016  017010016 
2 4c07d11e-0a3a-851d-e053-ca3ca8c0ca7f 017010046  017010046  017010046 
3 4c07d11e-0a4e-851d-e053-ca3ca8c0ca7f 017010037  017010037  017010037 
4 4c07d11e-0a25-851d-e053-ca3ca8c0ca7f 017010005  017010005  017010005 
5 4c07d11e-0a57-851d-e053-ca3ca8c0ca7f 017010036  017010036  017010036 
6 4c07d11e-2df2-851d-e053-ca3ca8c0ca7f 017010009  017010009  017010009 
  SA_GEOGID_ SA_CHANGE_ SA_URBAN_A SA_URBAN_1 geometry                       ⋯
1 A017010016 0          1          Carlow     MULTIPOLYGON (((673146.5 67... ⋯
2 A017010046 4          1          Carlow     MULTIPOLYGON (((673956.3 67... ⋯
3 A017010037 0          1          Carlow     MULTIPOLYGON (((673720.6 67... ⋯
4 A017010005 0          1          Carlow     MULTIPOLYGON (((671923.3 67... ⋯
5 A017010036 0          1          Carlow     MULTIPOLYGON (((673711.8 67... ⋯
6 A017010009 4          1          Carlow     MULTIPOLYGON (((673488.5 67... ⋯
  COUNTY_ENG COUNTY_GAE  CSO_LEA SHAPE_Leng SHAPE_Area
1 CARLOW     Ceatharlach CARLOW  2123.346   204944.2  
2 CARLOW     Ceatharlach CARLOW  2891.271   288225.4  
3 CARLOW     Ceatharlach CARLOW  2436.986   208426.9  
4 CARLOW     Ceatharlach CARLOW  2887.091   348798.6  
5 CARLOW     Ceatharlach CARLOW  2844.410   429349.1  
6 CARLOW     Ceatharlach CARLOW  1058.290    59299.2  
  geometry                       cellArea_m2 tcdArea_m2 tcdPerc treeCover 
1 MULTIPOLYGON (((673146.5 67... 204991.3      202.7830 0.099    0.7831503
2 MULTIPOLYGON (((673956.3 67... 288290.4      961.1264 0.333    0.7459016
3 MULTIPOLYGON (((673720.6 67... 208474.4      968.9902 0.465    0.7248895
4 MULTIPOLYGON (((671923.3 67... 348880.3    26616.5039 7.629   -0.4154939
5 MULTIPOLYGON (((673711.8 67... 429446.9    11372.1279 2.648    0.3773941
6 MULTIPOLYGON (((673488.5 67...  59312.7      107.7513 0.182    0.7699382

[1] "2025-01-25 17:44:29 GMT"

## Export

In [9]:
#output the data as a geojson
st_write(censusOA_TCD, file.path(pipelineDir, "oa_TCD.geojson"), delete_dsn = TRUE)

Deleting source `../../2_pipeline/Cork/1c_Copernicus/2022/oa_TCD.geojson' failed
Writing layer `oa_TCD' to data source 
  `../../2_pipeline/Cork/1c_Copernicus/2022/oa_TCD.geojson' using driver `GeoJSON'
Writing 18919 features with 32 fields and geometry type Multi Polygon.


**END**